In [1]:
from pexels_apis import PexelsAPI
import os
from dotenv import load_dotenv
import json
from react_agent.utils import extract_video_data
load_dotenv()

True

In [2]:
pexels = PexelsAPI(os.environ.get('PEXELS_API_KEY'))

In [3]:
search_params = {
    "query": "women sitting at the table staring at a document",
     
    "orientation": "portrait",
    "size": "medium",           
    "page": 1,                 
    "per_page": 1
}

# videos = pexels.search_videos(search_params)
# # json_output = json.dumps(videos, indent=4)
# # print(json_output)
# clean_data = extract_video_data(videos)
# from pprint import pprint
# pprint(clean_data)


In [5]:
# pexels.get_video(5520073)

In [6]:
# The JSON object representing the script
script_data = {
    "title": "Why You Feel Like a Fraud—Even When You're Winning",
    "length": "90-120 seconds",
    "background_music": {
        "music": "subtle motivational"
    },
    "sections": [
        {
            "section": "HOOK",
            "text": "You crushed the presentation. Got the promotion. Nailed the project. But deep down, you're waiting for someone to tap you on the shoulder and say, 'We made a mistake.'",
            "visual": {
                "scene": "woman sitting at desk staring at a 'Employee of the Month' certificate",
                "camera_angle": "close-up of conflicted facial expression",
                "transition": "cut to",
                "sound": {
                    "sound_effects": None,
                    "silence_duration": "1 second",
                    "sound_effect_timing": None
                }
            }
        },
        {
            "section": "CONCEPT",
            "text": "That voice whispering, 'You don’t belong here'? It’s not imposter syndrome—it’s your brain replaying old tapes. Tapes from a time when your worth was measured by what you did, not who you are.",
            "visual": {
                "scene": "man walking alone in a park, looking down at his feet",
                "camera_angle": "side profile in moody lighting",
                "transition": "fade to black",
                "sound": {
                    "sound_effects": None,
                    "silence_duration": None,
                    "sound_effect_timing": None
                }
            }
        },
        {
            "section": "REAL-WORLD EXAMPLE",
            "text": "You’re at a team meeting. Your idea gets praised. But instead of feeling proud, you think, 'They’re just being nice.' Later, you overhear a colleague say, 'She’s so confident.' You laugh bitterly. If only they knew. You replay every mistake, every hesitation, every moment you didn’t feel 'enough.' You work late—again—to prove you deserve the seat you’re already sitting in. But the more you achieve, the louder the doubt gets. It’s a loop: perform, doubt, overcompensate, repeat.",
            "visual": {
                "scene": "woman in a boardroom, colleagues clapping, but she looks uneasy",
                "camera_angle": "over-the-shoulder shot focusing on her discomfort",
                "transition": "cut to",
                "sound": {
                    "sound_effects": "faint office chatter",
                    "silence_duration": None,
                    "sound_effect_timing": "as the clapping starts"
                }
            }
        },
        {
            "section": "PSYCHOLOGICAL INSIGHT",
            "text": "This isn’t about your skills—it’s about internalized shame. You were conditioned to believe your value is tied to your output. So now, even when you succeed, your brain screams, 'Fraud!' because it doesn’t recognize worth without struggle.",
            "visual": {
                "scene": "person staring at their reflection in a window, looking lost",
                "camera_angle": "close-up of reflection with blurred background",
                "transition": "dissolve",
                "sound": {
                    "sound_effects": None,
                    "silence_duration": None,
                    "sound_effect_timing": None
                }
            }
        },
        {
            "section": "ACTIONABLE TIP",
            "text": "Start a 'proof of worth' journal. Every day, write one thing you did—not because you had to, but because you chose to. Not for praise, but for joy. Slowly, you’ll rewire your brain to see your value beyond productivity.",
            "visual": {
                "scene": "person writing in a notebook at a cozy café table",
                "camera_angle": "overhead shot of hands writing",
                "transition": "fade to black",
                "sound": {
                    "sound_effects": "pen scratching paper",
                    "silence_duration": None,
                    "sound_effect_timing": "as the writing begins"
                }
            }
        },
        {
            "section": "CTA",
            "text": "Follow for more psychology that helps you silence the fraud narrative—and start owning your worth.",
            "visual": {
                "scene": "person walking confidently down a city street, sunlight breaking through clouds",
                "camera_angle": "wide shot with uplifting lighting",
                "transition": "cut to black",
                "sound": {
                    "sound_effects": None,
                    "silence_duration": None,
                    "sound_effect_timing": None
                }
            }
        }
    ]
}

In [42]:
from pathlib import Path
import requests
import json
from typing import Dict
from react_agent.structures import VideoScript
from react_agent.utils import extract_video_data  # Adjust path as needed
import re

def sanitize_filename(name: str) -> str:
    """
    Convert a string to a safe filename:
    - Replace spaces with underscores
    - Remove non-alphanumeric characters (except underscores and dashes)
    - Collapse multiple underscores
    """
    name = name.replace(" ", "_")
    name = re.sub(r"[^\w\-]", "", name)  # Keep alphanumerics, underscores, dashes
    name = re.sub(r"_+", "_", name)  # Collapse multiple underscores
    return name.strip("_")


async def get_videos_from_script(script_data: Dict) -> Dict:
    """Search Pexels for videos matching each visual scene and download them."""

    try:
        script_obj = VideoScript(**script_data)
    except Exception as e:
        return {"status": "error", "message": f"Invalid script data: {e}"}

    # Create a sanitized directory for the video set
    safe_title = sanitize_filename(script_obj.title)
    video_dir = Path(f"my_test_files/videos/{safe_title}/")

    video_dir.mkdir(parents=True, exist_ok=True)

    results = []

    for section in script_obj.sections:
        search_query = section.visual.scene

        search_params = {
            "query": search_query,
            "orientation": "portrait",
            "size": "medium",
            "page": 1,
            "per_page": 1,
        }

        pexels_response = pexels.search_videos(search_params)  # Use actual object

        if pexels_response.get("status_code") != 200:
            print(f"Failed to find video for: {search_query}")
            continue

        videos_data = extract_video_data(pexels_response)
        if not videos_data:
            print(f"No videos returned for: {search_query}")
            continue

        # print(f'videos data: {videos_data}\n')
        best_video = videos_data[0]
        target_aspect = 9 / 16

        # Filter to SD quality portrait-oriented videos
        sd_videos = [
            v for v in best_video["video_files"]
            if v["quality"] == "sd" and v["width"] / v["height"] <= target_aspect
        ]
        if not sd_videos:
            sd_videos = [v for v in best_video["video_files"] if v["quality"] == "sd"]

        if not sd_videos:
            print(f"No suitable SD video found for: {search_query}")
            continue

        download_video = min(
            sd_videos, key=lambda v: abs((v["width"] / v["height"]) - target_aspect)
        )
        print(f'download video: {download_video}\n')
        # Generate sanitized filename
        filename = sanitize_filename(f"{safe_title}_{section.section}") + ".mp4"
        video_path = video_dir /  filename

        try:
            response = requests.get(download_video["link"], stream=True)
            response.raise_for_status()

            with open(video_path, "wb") as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)

            video_metadata = {
                "script_section": section.section,
                "pexels_id": best_video["id"],
                "author": best_video["author"],
                "author_url": best_video["author_url"],
                "video_url": best_video["video_url"],
                "download_url": download_video["link"],
                "file_path": str(video_path),
                "dimensions": f"{download_video['width']}x{download_video['height']}",
                "duration": best_video["duration"],
                "search_query": search_query,
                "attribution": f"Video by {best_video['author']} from Pexels",
            }

            results.append(video_metadata)

        except Exception as e:
            print(f"Failed to download video for {search_query}: {str(e)}")
            continue

    metadata_path = video_dir / "video_metadata.json"
    with open(metadata_path, "w") as f:
        json.dump(results, f, indent=4)

    return {
        "status": "success",
        "downloaded_videos": len(results),
        "total_sections": len(script_obj.sections),
        "video_directory": str(video_dir),
        "metadata_file": str(metadata_path),
        "videos": results,
    }


In [43]:
k = await get_videos_from_script(script_data)

download video: {'file_type': 'video/mp4', 'width': 226, 'height': 426, 'link': 'https://videos.pexels.com/video-files/8547568/8547568-sd_226_426_25fps.mp4', 'quality': 'sd', 'fps': 25.0, 'size': 567407}

download video: {'file_type': 'video/mp4', 'width': 226, 'height': 426, 'link': 'https://videos.pexels.com/video-files/6132796/6132796-sd_226_426_25fps.mp4', 'quality': 'sd', 'fps': 25.0, 'size': 961748}

download video: {'file_type': 'video/mp4', 'width': 360, 'height': 640, 'link': 'https://videos.pexels.com/video-files/8135918/8135918-sd_360_640_25fps.mp4', 'quality': 'sd', 'fps': 25.0, 'size': 759532}

download video: {'file_type': 'video/mp4', 'width': 226, 'height': 426, 'link': 'https://videos.pexels.com/video-files/7207576/7207576-sd_226_426_25fps.mp4', 'quality': 'sd', 'fps': 25.0, 'size': 899090}

download video: {'file_type': 'video/mp4', 'width': 240, 'height': 408, 'link': 'https://videos.pexels.com/video-files/4460555/4460555-sd_240_408_25fps.mp4', 'quality': 'sd', 'fps'

In [29]:
k

{'status': 'success',
 'downloaded_videos': 6,
 'total_sections': 6,
 'video_directory': 'my_test_files/videos/Why_You_Feel_Like_a_FraudEven_When_Youre_Winning',
 'metadata_file': 'my_test_files/videos/Why_You_Feel_Like_a_FraudEven_When_Youre_Winning/video_metadata.json',
 'videos': [{'script_section': 'HOOK',
   'pexels_id': 8547568,
   'author': 'Photo By: Kaboompics.com',
   'author_url': 'https://www.pexels.com/@karolina-grabowska',
   'video_url': 'https://www.pexels.com/video/a-tired-woman-looking-at-her-laptop-8547568/',
   'download_url': 'https://videos.pexels.com/video-files/8547568/8547568-sd_226_426_25fps.mp4',
   'file_path': 'my_test_files/videos/Why_You_Feel_Like_a_FraudEven_When_Youre_Winning/Why_You_Feel_Like_a_FraudEven_When_Youre_Winning_HOOK.mp4',
   'dimensions': '226x426',
   'duration': 19,
   'search_query': "woman sitting at desk staring at a 'Employee of the Month' certificate",
   'attribution': 'Video by Photo By: Kaboompics.com from Pexels'},
  {'script_sec

In [26]:
k['metadata_file']

'my_test_files/videos/Why_You_Feel_Like_a_FraudEven_When_Youre_Winning/video_metadata.json'

# ctc aligner

In [1]:
# import torch
# from ctc_forced_aligner import (
#     load_audio,
#     load_alignment_model,
#     generate_emissions,
#     preprocess_text,
#     get_alignments,
#     get_spans,
#     postprocess_results,
# )

# audio_path = "my_test_files/audios/one.wav"
# text_path = "my_test_files/audios/one.txt"
# language = "iso" # ISO-639-3 Language code
# device = "cuda" if torch.cuda.is_available() else "cpu"
# batch_size = 16


# alignment_model, alignment_tokenizer = load_alignment_model(
#     device,
#     dtype=torch.float16 if device == "cuda" else torch.float32,
# )

# audio_waveform = load_audio(audio_path, alignment_model.dtype, alignment_model.device)


# with open(text_path, "r") as f:
#     lines = f.readlines()
# text = "".join(line for line in lines).replace("\n", " ").strip()

# emissions, stride = generate_emissions(
#     alignment_model, audio_waveform, batch_size=batch_size
# )

# tokens_starred, text_starred = preprocess_text(
#     text,
#     romanize=True,
#     language=language,
# )

# segments, scores, blank_token = get_alignments(
#     emissions,
#     tokens_starred,
#     alignment_tokenizer,
# )

# spans = get_spans(tokens_starred, segments, blank_token)

# word_timestamps = postprocess_results(text_starred, spans, stride, scores)

# kokoro-onnx

In [ ]:

import soundfile as sf
from misaki import en, espeak

from kokoro_onnx import Kokoro

# Misaki G2P with espeak-ng fallback
fallback = espeak.EspeakFallback(british=False)
g2p = en.G2P(trf=False, british=False, fallback=fallback)

# Kokoro
kokoro = Kokoro("/home/aditya-ladawa/Aditya/z_projects/short_creation/kokoro_files/kokoro.onnx", "/home/aditya-ladawa/Aditya/z_projects/short_creation/kokoro_files/voices-v1.0.bin")

# Phonemize
text = "This is the fading affect bias, we remember the good times more vividly than the slow drift apart. It’s also about emotional inertia: relationships need momentum to survive. Without it, they fade, not with a bang, but with a whisper."
phonemes, _ = g2p(text)

# Create
samples, sample_rate = kokoro.create(phonemes, "af_jessica", is_phonemes=True)

# Save
sf.write("audio.wav", samples, sample_rate)
print("Created audio.wav")

Created audio.wav


In [3]:
# ! wget https://github.com/thewh1teagle/kokoro-onnx/releases/download/model-files-v1.0/voices-v1.0.bin